# Ensembling already trained models

Setup & Config

In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

import os 
import tensorflow as tf
from pathlib import Path
import cv2

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:03:00.0, compute capability: 7.5



Reinitialize benchmark that was used for training:

In [2]:
columns_12 = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']

uzeros = ['Cardiomegaly',
        'Enlarged Cardiomediastinum',
        'Lung Opacity',
        'Lung Lesion',
        'Consolidation',
        'Pneumothorax',
        'Pleural Effusion']
uones = ['Edema',
        'Atelectasis',
        'Fracture',
        'Pleural Other',
        'Pneumonia',]

upsample_factors = {
    "Enlarged Cardiomediastinum": 1,
    "Lung Lesion":1,
    "Pleural Other":2,
    "Fracture":2,
}

transformations_2 = {"unsharp_mask":{"radius":2, "amount":1}}

benchmark_params = {"path":Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                             "name":"2_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled",
                                             "classes":columns_12,
                                             "train_labels" : "train.csv",
                                             "test_labels" : "test.csv",
                                             "nan_replacement" : 0, #float("NaN"),
                                             "u_enc" : [uzeros, uones],
                                             "epochs":5,
                                             "batch_size":32,
                                             "crop" : False,
                                             "dim":(256, 256),
                                             "loss" : BinaryCrossentropy(),
                                             "use_class_weights" : False,
                                             "upsample_factors" : upsample_factors,
                                             "metrics":METRICS,
                                             "single_class_metrics":SINGLE_CLASS_METRICS,
                                             "optimizer" : Adam(learning_rate=1e-4, clipnorm=1),
                                             "lr_factor" : 0.1,
                                             "augmentation" : "affine",
                                             "transformations" : transformations_2,
                                             "split_seed" : 6122156,
                                             "split_valid_size" : 0.05}

Initialzed custom_binary_crossentropy with epsilon 1e-05


Specify which models you want to ensemble:

In [3]:
import tensorflow
from tensorflow.keras.applications.densenet import DenseNet169, DenseNet121
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

ensemble = [[DenseNet169, tensorflow.keras.applications.densenet.preprocess_input, 'models/DenseNet169_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled/weights.05-0.21.hdf5'],
 [DenseNet121, tensorflow.keras.applications.densenet.preprocess_input, 'models/DenseNet121_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled/weights.05-0.21.hdf5'],
 [Xception, tensorflow.keras.applications.xception.preprocess_input, 'models/Xception-2_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled/weights.05-0.21.hdf5'],
 [InceptionResNetV2, tensorflow.keras.applications.inception_resnet_v2.preprocess_input, 'models/InceptionResNetV2_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled/weights.05-0.21.hdf5']]

In [4]:
predictions = []

Next, we ensemble the model and make the predictions.

In [11]:
member = ensemble[3]
benchmark_params['preprocess_input_fn'] = member[1]
bce_benchmark = Chexpert_Benchmark (**benchmark_params)
bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
                                                    bce_benchmark.negative_weights)

bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, member[0], bce_benchmark.label_columns)
bce_chexpert_exp.benchmark.testgen.shuffle = False
print(bce_chexpert_exp.benchmark.testgen.shuffle)

bce_chexpert_exp = Experiment(bce_benchmark, bce_chexpert_exp.model)
bce_chexpert_exp.model.load_weights(member[2])
predictions.append(bce_chexpert_exp.model.predict(bce_chexpert_exp.benchmark.testgen))
    

False


In [12]:
print(len(predictions))

4


We take the average over all predictions and evaluate them with the known metrics:

In [13]:
from sklearn.metrics import classification_report

y_pred = tf.math.reduce_mean(predictions, axis=0)
predictions_bool = (y_pred >= 0.5)
groundtruth_label = bce_chexpert_exp.benchmark.testgen.get_labels_nonan()

for key, value in enumerate(columns_12):
    pr = []
    for sample in y_pred:
        pr.append(sample[key])
    m = tf.keras.metrics.AUC()
    m.update_state(groundtruth_label[:, key], pr)
    print(value, m.result().numpy())
    
m = tf.keras.metrics.AUC()
m.update_state(groundtruth_label, y_pred)
print("Average AUC: ", m.result().numpy())

predictions_bool = np.array(predictions_bool, dtype=int)
groundtruth_label = bce_chexpert_exp.benchmark.testgen.get_labels_nonan()
report = classification_report(groundtruth_label, predictions_bool, target_names=bce_chexpert_exp.benchmark.label_columns)
print('skelarn report: ', report)

Enlarged Cardiomediastinum 0.39922935
Cardiomegaly 0.6671687
Lung Opacity 0.90432096
Lung Lesion 0.31545064
Edema 0.9021164
Consolidation 0.93057436
Pneumonia 0.80503315
Atelectasis 0.8431818
Pneumothorax 0.7859514
Pleural Effusion 0.9248816
Pleural Other 0.80686694
Fracture 0.0
Average AUC:  0.6658534
skelarn report:                              precision    recall  f1-score   support

Enlarged Cardiomediastinum       1.00      0.01      0.02       109
              Cardiomegaly       0.00      0.00      0.00        68
              Lung Opacity       0.00      0.00      0.00       126
               Lung Lesion       0.00      0.00      0.00         1
                     Edema       0.73      0.53      0.62        45
             Consolidation       0.00      0.00      0.00        33
                 Pneumonia       0.50      0.12      0.20         8
               Atelectasis       0.00      0.00      0.00        80
              Pneumothorax       0.25      0.38      0.30         

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1